# Matrix-vector multiplication via <code>axpy</code>s

This notebook walks you through how to implement $ y := A x + y $ via axpy operations.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with `include` and `using` statements.

## Algorithm

<image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/342Mvmult_n_unb_var2.png" alt="Matrix-vector multiplication via dot products algorithm" width="75%">

## The `Mvmult_n_unb_var2!( A, x, y )` routine

This routine, given $ A \in \mathbb{R}^{m \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^m $, computes $ y := A x + y $.  The "_n_" indicates this is the "no transpose" matrix-vector multiplication.  What this means will become clear next week.

The specific laff function we will use is 
<ul>
<li> <code> laff.axpy!( alpha, x, y ) </code> which computes $ y := \alpha x + y $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage </a>.

In [5]:
include("../flame.jl")
using .flame
include("../laff/laff.jl")
using .laff

function Mvmult_n_unb_var2!(A, x, y)

    AL, AR = flame.part_1x2(A,
                            0, "LEFT")

    xT,
    xB  = flame.part_2x1(x,
                         0, "TOP")

    while size(AL, 2) < size(A, 2)

        A0, a1, A2 = flame.repart_1x2_to_1x3(AL, AR,
                                             1, "RIGHT")

        x0,
        chi1,
        x2    = flame.repart_2x1_to_3x1(xT,
                                        xB,
                                        1, "BOTTOM")

        #------------------------------------------------------------#

        laff.axpy!( chi1, a1, y )

        #------------------------------------------------------------#

        AL, AR = flame.cont_with_1x3_to_1x2(A0, a1, A2,
                                            "LEFT")

        xT,
        xB  = flame.cont_with_3x1_to_2x1(x0,
                                         chi1,
                                         x2,
                                         "TOP")
    end
end






Mvmult_n_unb_var2! (generic function with 1 method)

## Testing

Let's quickly test the routine by creating a 4 x 3 matrix and related vectors, performing the computation.

In [6]:
A = rand(4, 3)
x = rand(3)
y = rand(4)
yold = rand(4)

println("y before =")
y

y before =


4-element Array{Float64,1}:
 0.5625150969094053 
 0.1311468297266889 
 0.0595815728127258 
 0.25362793577593945

In [7]:
println("x before =")
x

x before =


3-element Array{Float64,1}:
 0.683188187756594   
 0.38135818692975554 
 0.031997715508811586

In [8]:
laff.copy!( y, yold )   # save the original vector y

Mvmult_n_unb_var2!( A, x, y )

println( "y after = " )
y

y after = 


4-element Array{Float64,1}:
 1.269818186102787 
 0.6641267322363824
 0.9224671576669761
 0.9984809520258291

In [9]:
println( "y - ( A * x + yold ) = " )
y - ( A * x + yold )

y - ( A * x + yold ) = 


4-element Array{Float64,1}:
 -2.220446049250313e-16 
  1.1102230246251565e-16
  0.0                   
  0.0                   

Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.